In [1]:
# Exercício: Breast Cancer Wisconsin (Diagnostic) - UCI
# Requisitos: pandas, numpy, requests (opcional), matplotlib (não é necessário aqui)

import pandas as pd
import numpy as np

In [2]:
# ---------------------------------------------------------------------
# 1) Carregamento e inspeção inicial
# ---------------------------------------------------------------------
# URL direta do arquivo de dados no UCI:
DATA_URL = "https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data"

# Nomes de colunas (32 no total: id, diagnosis + 30 atributos)
feature_groups = ["mean", "se", "worst"]
base_features = [
    "radius", "texture", "perimeter", "area", "smoothness",
    "compactness", "concavity", "concave_points", "symmetry",
    "fractal_dimension"
]
feature_cols = [f"{b}_{g}" for g in feature_groups for b in base_features]
cols = ["id", "diagnosis"] + feature_cols  # total 32

# Carrega CSV sem header e atribui nomes
df = pd.read_csv(DATA_URL, header=None, names=cols)

# a) & b) & c)
print("Dimensões (linhas, colunas):", df.shape)
print("\nPrimeiras 5 linhas:")
print(df.head())

Dimensões (linhas, colunas): (569, 32)

Primeiras 5 linhas:
         id diagnosis  radius_mean  texture_mean  perimeter_mean  area_mean  \
0    842302         M        17.99         10.38          122.80     1001.0   
1    842517         M        20.57         17.77          132.90     1326.0   
2  84300903         M        19.69         21.25          130.00     1203.0   
3  84348301         M        11.42         20.38           77.58      386.1   
4  84358402         M        20.29         14.34          135.10     1297.0   

   smoothness_mean  compactness_mean  concavity_mean  concave_points_mean  \
0          0.11840           0.27760          0.3001              0.14710   
1          0.08474           0.07864          0.0869              0.07017   
2          0.10960           0.15990          0.1974              0.12790   
3          0.14250           0.28390          0.2414              0.10520   
4          0.10030           0.13280          0.1980              0.10430   

  

In [3]:
# ---------------------------------------------------------------------
# 2) Estatísticas descritivas básicas
# ---------------------------------------------------------------------
vars_basic = ["radius_mean", "texture_mean", "area_mean"]

# a) média, mediana, desvio-padrão
desc_stats = pd.DataFrame({
    "mean": df[vars_basic].mean(),
    "median": df[vars_basic].median(),
    "std": df[vars_basic].std(ddof=1)
})
print("\nEstatísticas básicas (mean/median/std):")
print(desc_stats)

# b) quartis e IQR
q1 = df[vars_basic].quantile(0.25)
q2 = df[vars_basic].quantile(0.50)  # mediana (Q2)
q3 = df[vars_basic].quantile(0.75)
iqr = q3 - q1
quartis_iqr = pd.DataFrame({"Q1": q1, "Q2": q2, "Q3": q3, "IQR": iqr})
print("\nQuartis e IQR:")
print(quartis_iqr)



Estatísticas básicas (mean/median/std):
                    mean  median         std
radius_mean    14.127292   13.37    3.524049
texture_mean   19.289649   18.84    4.301036
area_mean     654.889104  551.10  351.914129

Quartis e IQR:
                  Q1      Q2      Q3     IQR
radius_mean    11.70   13.37   15.78    4.08
texture_mean   16.17   18.84   21.80    5.63
area_mean     420.30  551.10  782.70  362.40


In [4]:
# ---------------------------------------------------------------------
# 3) Análise por grupo
# ---------------------------------------------------------------------
# a) média de radius_mean por diagnóstico (B vs M)
mean_radius_by_dx = df.groupby("diagnosis")["radius_mean"].mean()
print("\nMédia de radius_mean por diagnóstico:")
print(mean_radius_by_dx)

# b) comparar média de area_worst entre benignos e malignos
mean_area_worst_by_dx = df.groupby("diagnosis")["area_worst"].mean()
print("\nMédia de area_worst por diagnóstico:")
print(mean_area_worst_by_dx)



Média de radius_mean por diagnóstico:
diagnosis
B    12.146524
M    17.462830
Name: radius_mean, dtype: float64

Média de area_worst por diagnóstico:
diagnosis
B     558.899440
M    1422.286321
Name: area_worst, dtype: float64


In [3]:
# ---------------------------------------------------------------------
# 4) Distribuições e frequências
# ---------------------------------------------------------------------
# a) value_counts absoluto e proporcional
counts_abs = df["diagnosis"].value_counts()
counts_pct = df["diagnosis"].value_counts(normalize=True) * 100
print("\nDistribuição do diagnóstico (absoluto):")
print(counts_abs)
print("\nDistribuição do diagnóstico (%):")
print(counts_pct.round(2))

# b) tabela com n e % lado a lado
tab_dx = pd.DataFrame({
    "n": counts_abs,
    "%": counts_pct.round(2)
})
print("\nTabela n e % lado a lado:")
print(tab_dx)


Distribuição do diagnóstico (absoluto):
diagnosis
B    357
M    212
Name: count, dtype: int64

Distribuição do diagnóstico (%):
diagnosis
B    62.74
M    37.26
Name: proportion, dtype: float64

Tabela n e % lado a lado:
             n      %
diagnosis            
B          357  62.74
M          212  37.26


In [4]:
# ---------------------------------------------------------------------
# 5) Correlação e filtros
# ---------------------------------------------------------------------
# a) correlação entre radius_mean e perimeter_mean
corr_radius_perimeter = df["radius_mean"].corr(df["perimeter_mean"])
print("\nCorrelação radius_mean vs perimeter_mean:", corr_radius_perimeter)

# b) 3 variáveis mais correlacionadas com o diagnóstico (em módulo)
#    converter diagnosis para binária: M=1, B=0
df["_diagnosis_bin"] = (df["diagnosis"] == "M").astype(int)

# correlação de todas as variáveis numéricas com a coluna binária
num_cols = df.select_dtypes(include=[np.number]).columns.drop(["id"])
corr_with_dx = df[num_cols].corr()["_diagnosis_bin"].drop("_diagnosis_bin")

top3 = corr_with_dx.abs().sort_values(ascending=False).head(3)
top3_with_sign = corr_with_dx.loc[top3.index]
print("\nTop-3 variáveis mais correlacionadas com o diagnóstico (|r|):")
print(pd.DataFrame({
    "corr": top3_with_sign.values,
    "|corr|": top3.values
}, index=top3.index))

# c) 10 casos com maior radius_worst (mostrar diagnosis, radius_worst, area_worst)
top10 = df.nlargest(10, "radius_worst")[["diagnosis", "radius_worst", "area_worst"]]
print("\n10 casos com maior radius_worst:")
print(top10.to_string(index=False))


Correlação radius_mean vs perimeter_mean: 0.9978552814938112

Top-3 variáveis mais correlacionadas com o diagnóstico (|r|):
                          corr    |corr|
concave_points_worst  0.793566  0.793566
perimeter_worst       0.782914  0.782914
concave_points_mean   0.776614  0.776614

10 casos com maior radius_worst:
diagnosis  radius_worst  area_worst
        M         36.04      4254.0
        M         33.13      3234.0
        M         33.12      3216.0
        M         32.49      3432.0
        M         31.01      2944.0
        M         30.79      2782.0
        M         30.75      3143.0
        M         30.67      2906.0
        M         30.00      2562.0
        M         29.92      2642.0
